In [17]:
import pandas as pd
import numpy as np
import pyspark as ps
from pyspark.sql import SparkSession
from pyspark.sql.types import (
    IntegerType, StringType, IntegerType, FloatType, 
    StructField, StructType, DoubleType
)
from pyspark.ml.recommendation import ALS
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from pyspark.sql.functions import udf, col, isnan
import psycopg2
from pandas.io import sql
from sklearn.model_selection import train_test_split
from pyspark.ml.evaluation import RegressionEvaluator
from itertools import product

In [18]:
cd ../mixmaker/

/Users/brettashley/galvanize/dsi/00-Capstone/ml-mix-lab/mixmaker


In [19]:
import database_interaction
import model

In [20]:
df = db.get_table('connections_distinct')
data = sr.spark.createDataFrame(df)

In [23]:
train, test = data.randomSplit([0.8, 0.2])

In [24]:
sr = model.SongRecommender(itemCol='song_id',
                     userCol='sampled_by_song_id',
                     ratingCol='is_connected',
                     nonnegative=True,
                     regParam=0.01,
                     rank=100,
                     alpha=30,
                     implicitPrefs=True)

db = database_interaction.DatabaseInteraction(db_name='mixmaker')

In [25]:
recommender = sr.fit(train)

In [26]:
preds = sr.transform(recommender, test)

In [38]:
(preds.filter("prediction + 1 > prediction")
      .filter("song_id != sampled_by_song_id")
      .sort(col("prediction")
      .desc()).limit(200).show())

+-------+------------------+------------+-------+----------+
|song_id|sampled_by_song_id|is_connected|checked|prediction|
+-------+------------------+------------+-------+----------+
|  27951|              9129|           1|    1.0|  1.789589|
|  13951|             14540|           1|    NaN| 1.7865506|
|   6874|             14540|           1|    NaN| 1.7855539|
|      3|               121|           1|    1.0| 1.7478492|
|   1776|              6308|           1|    NaN| 1.7300161|
|  18325|              8280|           1|    1.0|  1.621242|
|    600|                 2|           1|    NaN| 1.5344161|
|   2353|                24|           1|    NaN| 1.5106838|
|  12428|             12576|           1|    1.0| 1.4740473|
|  10784|             10932|           1|    1.0| 1.4228543|
|  13755|             24853|           1|    NaN|  1.302945|
|  44159|             14540|           1|    NaN| 1.2959192|
|   8280|             18325|           1|    NaN| 1.2754506|
|   6112|             14

In [27]:
sr.rmse(preds, True)

0.9356162860256314

In [39]:
test_with_zeroes = sr.generate_negative_targets(test,
                                    'song_id', 'sampled_by_song_id', 'is_connected')

DataFrame[index: bigint, checked: double, is_connected: bigint, sampled_by_song_id: bigint, song_id: bigint]

In [46]:
preds_with_zeroes = sr.transform(recommender, test_with_zeros)

In [47]:
preds_with_zeroes.show()

+-----+-------+------------+------------------+-------+------------+
|index|checked|is_connected|sampled_by_song_id|song_id|  prediction|
+-----+-------+------------+------------------+-------+------------+
|11722|    NaN|           0|             15865|    148|5.0680275E-5|
|13526|    NaN|           0|             29055|    148| 0.027482284|
| 1499|    1.0|           1|              4812|    148| 0.028303787|
| 2108|    NaN|           1|              1088|   1088|         0.0|
| 2185|    NaN|           1|              1238|   1238|         NaN|
|11032|    NaN|           0|              1261|   1238| 0.005694208|
|23242|    NaN|           0|             14821|   1238|         0.0|
|11158|    NaN|           0|             22245|   1238|         0.0|
| 1621|    NaN|           0|             15349|   1238|         0.0|
|22370|    NaN|           0|              9304|   1238|2.0392449E-4|
| 2460|    NaN|           1|              1645|   1645| 0.056136895|
| 2459|    NaN|           1|      

In [48]:
sr.rmse(preds_with_zeroes, True)

0.6540079056090256

In [ ]:
preds_train = sr.transform(recommender, train)

In [ ]:
sr.rmse(preds_train)

In [ ]:
preds.filter("prediction + 1 > prediction").count()

In [ ]:
preds.count()

In [ ]:
top_10_recs = recommender.recommendForAllUsers(10).toPandas()

In [ ]:
len(top_10_recs)

In [ ]:
train.count()

In [ ]:
top_10_recs.head()

In [ ]:
top_10_recs.loc[0,'recommendations']

In [ ]:
for i, row in top_10_recs.head().iterrows():
    print(list([row[0]] * 10))
    print((pd.DataFrame(row['recommendations'])))

In [ ]:
output = pd.DataFrame(columns=['user_song_id', 'item_song_id', 'rating'])

In [ ]:
output

In [ ]:
for i, row in top_10_recs.head().iterrows():
    df = pd.DataFrame(row[1], columns=['item_song_id', 'rating'])
    df['user_song_id'] = list([row[0]] * 10)
    output = pd.concat([output, df])

In [ ]:
recommender = sr.fit(data)

In [ ]:
preds = sr.get_predictions_for_all_users(recommender)

In [ ]:
db = database_interaction.DatabaseInteraction()

In [ ]:
preds.columns

In [ ]:
','.join((preds.loc[:,'user_song_id'].values))

In [ ]:
query = """
        INSERT INTO predictions_temp (user_song_id, item_song_id, rating)
        SELECT %s, %s, %s
        """

In [ ]:
for i, row in preds.head().iterrows():
    if i % 2 == 0:
        print(i)
    print(i, row[0], row[1], row[2])

In [ ]:
test = preds[:100].reset_index(drop=True)

In [ ]:
preds_reset = preds.reset_index(drop=True)

In [ ]:
len(preds_reset)

In [ ]:
for row in test.itertuples():
    if row[0] % 10 == 0:
        print(row[0])
    print(row[0], row[1], row[2], row[3])

In [ ]:
for row in preds_reset.head().itertuples():
    if row[0] % 5000 == 0:
        print(row[0])
    db.cur.execute(query, 
                   (row[1], row[2], row[3]))
    db.conn.commit()

In [ ]:
for row in preds_reset.itertuples():
    if row[0] % 5000 == 0:
        print(row[0])
    db.cur.execute(query, 
                   (row[1], row[2], row[3]))
    db.conn.commit()

In [ ]:
df = data.toPandas()

In [ ]:
df.columns

In [ ]:
existing_pairs = set((x,y) for 
                     [x,y] in df[['song_id', 'sampled_by_song_id']].values)

In [ ]:
len(existing_pairs)

In [ ]:
for row in test.itertuples():
    if (row[1], row[2]) in existing_pairs:
        test.drop(row[0], inplace=True)

In [ ]:
test

In [ ]:
sr = model.SongRecommender()

In [ ]:
train, test = data.randomSplit([.995, .005])

In [ ]:
recommender = sr.fit(test)

In [ ]:
sr.get_predictions_for_all_users(data, recommender, 10)

In [ ]:
output = _

In [ ]:
output.loc[output['user_song_id'] == output['item_song_id']]

In [ ]:
recs = recommender.recommendForAllUsers(50)

In [ ]:
recs_df = recs.toPandas()

In [ ]:
output = pd.DataFrame(
    columns=['user_song_id', 'item_song_id', 'rating'])

In [ ]:
for row in recs_df.head().itertuples():
    df = pd.DataFrame(row[2], columns=['item_song_id', 'rating'])
    df['user_song_id'] = list([row[1]] * (10 * 5))
    for entry in df.itertuples():
        print(entry[3], entry[1])
        if (entry[3], entry[1]) in existing_pairs:
            df.drop(row[0], axis=0, inplace=True)
    output = pd.concat([output, df[:10]], sort=True)

In [ ]:
output.loc[output['user_song_id'] == output['item_song_id']]

In [ ]:
(22970, 22970) in existing_pairs

In [49]:
cd '/Users/brettashley/galvanize/dsi/00-Capstone/ml-mix-lab/mixmaker/'

/Users/brettashley/galvanize/dsi/00-Capstone/ml-mix-lab/mixmaker


In [3]:
import model

In [4]:
import database_interaction

In [11]:
    sr = model.SongRecommender(itemCol='song_id',
                        userCol='sampled_by_song_id',
                        ratingCol='is_connected',
                        nonnegative=True,
                        regParam=0.01,
                        rank=200,
                        alpha=1)

db = database_interaction.DatabaseInteraction(db_name='mixmaker')

df = db.get_table('connections_distinct')
data = sr.spark.createDataFrame(df)
recommender = sr.fit(data)

In [12]:
preds = sr.get_predictions_for_all_users(df, recommender, n_predictions=10)

In [13]:
db.write_predictions(preds, 'predictions')

0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0

0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0

0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0

0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0

0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0

...

0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0

0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0

0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0

0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0

0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831
0/398831


In [50]:
import get_predictions

In [53]:
get_predictions.get_and_write_predictions()

Getting data...
Fitting model...
Getting predictions...
0
10000
20000
30000
Writing predictions...
